# Before we Begin downloading our module, let us create a more Keras friendly environment 

In [1]:
conda create --name keras_env

Solving environment: done

## Package Plan ##

  environment location: /home/wsuser/.conda/envs/keras_env



Preparing transaction: done
Verifying transaction: done
Executing transaction: done
#
# To activate this environment, use
#
#     $ conda activate keras_env
#
# To deactivate an active environment, use
#
#     $ conda deactivate


Note: you may need to restart the kernel to use updated packages.


In [2]:
conda activate keras_env


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.



Note: you may need to restart the kernel to use updated packages.


In [3]:
conda install -c anaconda keras

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


# Let us import the required libraries

In [4]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input

## Now that we have created our Keras environment, let us download the data

In [5]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2021-07-05 09:06:26--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip.6’

concrete_data_week4 100%[===================>] 249.37M  36.2MB/s    in 7.0s    

2021-07-05 09:06:33 (35.4 MB/s) - ‘concrete_data_week4.zip.6’ saved [261483817/261483817]



In [6]:
!unzip -q concrete_data_week4.zip

[concrete_data_week4.zip]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of concrete_data_week4.zip or
        concrete_data_week4.zip.zip, and cannot find concrete_data_week4.zip.ZIP, period.


## Define Global Constants

Here, we will define constants that we will be using throughout the rest of the lab. 

1.  We are obviously dealing with two classes, so _num_classes_ is 2. 
2.  The ResNet50 model was built and trained using images of size (224 x 224). Therefore, we will have to resize our images from (227 x 227) to (224 x 224).
3.  We will training and validating the model using batches of 100 images.


In [8]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

In [9]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


## Build, Compile and Fit Model

In [10]:
model = Sequential()

In [11]:
model.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

94773248/94765736 [==============================] - 3s 0us/step


In [12]:
model.add(Dense(num_classes, activation='softmax'))

In [13]:
model.layers

In [14]:
model.layers[0].layers

In [15]:
model.layers[0].trainable = False

In [16]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [18]:
fit_history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Epoch 1/2
301/301 [==============================] - 6789s 23s/step - loss: 0.0336 - accuracy: 0.9873 - val_loss: 0.0066 - val_accuracy: 0.9986
Epoch 2/2
301/301 [==============================] - 6739s 22s/step - loss: 0.0054 - accuracy: 0.9987 - val_loss: 0.0049 - val_accuracy: 0.9989


In [ ]:
model.save('classifier_resnet_model.h5')